## Script is designed to create the taxonomy table file with NCBI tax-id

Input is: <br>

1. <i> File "Tax_gtdb-ncbi.csv" <br></i>
Taxonomy table file. The table includes the gtdb taxonomy and ncbi names for all species for which I find a correspondence between dtdb and ncbi spesies names. 
Gtdb species names vs ncbi species names match files were taken from here: <br>
https://data.gtdb.ecogenomic.org/releases/latest/auxillary_files/

2. <i> File "names.dmp" <br></i>
File with ncbi name and ncbi tax-id. From zip folder: ncbi_taxdump_20220917 <br>
https://data.gtdb.ecogenomic.org/releases/latest/auxillary_files/ <br>
<br>


In [2]:
# import packages
import pandas as pd

## Assign NCBI tax_id to GTDB species

In [40]:
# read file with GTDB and NCBI taxonomies mach from "NCBI_vs_GTDB" scripts folder
rel='v220'
df1 = pd.read_excel('/Users/katerynapantiukh/Documents/1MyDisk/Code/Microbiome/NCBI_vs_GTDB/results/Tax_gtdb-ncbi_'+str(rel)+'.xlsx')
# read "names.dmp" file from NCBI taxonomy
name = pd.read_csv('/Users/katerynapantiukh/Documents/1MyDisk/Code/Microbiome/NCBI_tax_id/input/names.dmp', sep = '\t', header = None)
name.columns = ['tax_id','1', 'name_txt', '3','unique_name','5','name_class','7']
name = name[name['name_class'] == 'scientific name']
name = name.drop(columns = ['name_class', 'unique_name','1', '3', '5', '7'])
df2 = name.reset_index(drop = True)
df2.head(5)


,tax_id,name_txt
0,1,root
1,2,Bacteria
2,6,Azorhizobium
3,7,Azorhizobium caulinodans
4,9,Buchnera aphidicola


In [44]:
# add tax-id to the table
# Function to extract all species names from sp_ncbi
def extract_species_names(sp_ncbi):
    if pd.isna(sp_ncbi):
        return []
    species_names = []
    for name in sp_ncbi.split(','):
        parts = name.strip().split()
        if len(parts) >= 2:
            species_names.append(parts[0] + " " + parts[1])
    return species_names

# Expand sp_ncbi to multiple rows for each species name
expanded_rows = []
for idx, row in df1.iterrows():
    species_names = extract_species_names(row['sp_ncbi'])
    for name in species_names:
        expanded_rows.append([row['sp_gtdb'], name])

expanded_df = pd.DataFrame(expanded_rows, columns=['sp_gtdb', 'species_name'])

# Merge expanded_df with df2
merged_df = pd.merge(expanded_df, df2, how='left', left_on='species_name', right_on='name_txt')

# Group by sp_gtdb and aggregate tax_ids
grouped_df = merged_df.groupby('sp_gtdb')['tax_id'].apply(lambda x: ', '.join(x.dropna().astype(str))).reset_index()

# Merge the aggregated tax_ids back to the original df1
final_df = pd.merge(df1, grouped_df, how='left', on='sp_gtdb')

final_df.to_excel('results/TaxID_gtdb-ncbi_'+str(rel)+'.xlsx', index=False)
final_df


,domain,phylum,class,order,family,genus,sp,sp_gtdb,sp_ncbi,tax_id
0,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter,Methanobrevibacter sp017432245,Methanobrevibacter sp017432245,NaN,NaN
1,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter,Methanobrevibacter sp017410345,Methanobrevibacter sp017410345,NaN,NaN
2,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter,Methanobrevibacter sp017646885,Methanobrevibacter sp017646885,NaN,NaN
3,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter,Methanobrevibacter sp024409185,Methanobrevibacter sp024409185,NaN,NaN
4,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter,Methanobrevibacter sp900314635,Methanobrevibacter sp900314635,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
19690,Bacteria,Pseudomonadota,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Stutzerimonas,Stutzerimonas zhaodongensis,Stutzerimonas zhaodongensis,[Pseudomonas] zhaodongensis,1176257.0
19691,Bacteria,Actinomycetota,Actinomycetes,Propionibacteriales,Propionibacteriaceae,Arachnia,Arachnia massiliensis,Arachnia massiliensis,[Pseudopropionibacterium] massiliense,2220000.0
19692,Bacteria,Bacillota_A,Clostridia,Lachnospirales,Lachnospiraceae,Ruminococcus_B,Ruminococcus_B gnavus,Ruminococcus_B gnavus,"[Ruminococcus] gnavus 91.75%, (g__)",33038.0
19693,Bacteria,Bacillota_A,Clostridia,Lachnospirales,Lachnospiraceae,Mediterraneibacter,Mediterraneibacter lactaris,Mediterraneibacter lactaris,"[Ruminococcus] lactaris 81.97%, (g__)",46228.0


### Random check merge table

In [45]:
final_df.loc[final_df['sp_gtdb'] == 'Odoribacter splanchnicus']

,domain,phylum,class,order,family,genus,sp,sp_gtdb,sp_ncbi,tax_id
15723,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Marinifilaceae,Odoribacter,Odoribacter splanchnicus,Odoribacter splanchnicus,"Odoribacter splanchnicus 89.15%, (g__)",28118.0


In [38]:
print('For db version',rel)
print('Total GTDB species in initial table is:',len(df1))
print('Total NCBI species in initial table  is:',len(df2))
print('Total species in GTDB-NCBI result table is ',len(final_df))

For db version v214
Total GTDB species in initial table is: 12214
Total NCBI species in initial table  is: 2442982
Total species in GTDB-NCBI result table is  12214


In [46]:
print('For db version',rel)
print('Total GTDB species in initial table is:',len(df1))
print('Total NCBI species in initial table  is:',len(df2))
print('Total species in GTDB-NCBI result table is ',len(final_df))

For db version v220
Total GTDB species in initial table is: 19695
Total NCBI species in initial table  is: 2442982
Total species in GTDB-NCBI result table is  19695
